In [24]:
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
# Use AerSimulator from qiskit_aer
from qiskit_aer import AerSimulator
from qiskit.visualization import plot_histogram
import matplotlib.pyplot as plt # Import for displaying plots

# --- Common Setup ---
# Define registers
data = QuantumRegister(5, 'd')    # D1, D2, D3, D4, D5
ancilla = QuantumRegister(4, 'a') # A1, A2, A3, A4
# Use ONE classical register for syndrome measurement in each circuit
syndrome = ClassicalRegister(4, 's')

# Define a function to measure stabilizers (to avoid repetition)
def measure_stabilizers(qc, data_reg, ancilla_reg, syndrome_reg):
    """Applies stabilizer measurements to the circuit."""
    # S1 = X_D1 X_D2 X_D3 (measures onto syndrome_reg[0])
    qc.h(ancilla_reg[0])
    qc.cx(ancilla_reg[0], data_reg[0])
    qc.cx(ancilla_reg[0], data_reg[1])
    qc.cx(ancilla_reg[0], data_reg[2])
    qc.h(ancilla_reg[0])
    qc.measure(ancilla_reg[0], syndrome_reg[0])
    qc.reset(ancilla_reg[0]) # Reset ancilla for next measurement

    # S2 = Z_D1 Z_D3 Z_D4 (measures onto syndrome_reg[1])
    qc.cx(data_reg[0], ancilla_reg[1])
    qc.cx(data_reg[2], ancilla_reg[1])
    qc.cx(data_reg[3], ancilla_reg[1])
    qc.measure(ancilla_reg[1], syndrome_reg[1])
    qc.reset(ancilla_reg[1]) # Reset ancilla

    # S3 = Z_D2 Z_D3 Z_D5 (measures onto syndrome_reg[2])
    qc.cx(data_reg[1], ancilla_reg[2])
    qc.cx(data_reg[2], ancilla_reg[2])
    qc.cx(data_reg[4], ancilla_reg[2])
    qc.measure(ancilla_reg[2], syndrome_reg[2])
    qc.reset(ancilla_reg[2]) # Reset ancilla

    # S4 = X_D3 X_D4 X_D5 (measures onto syndrome_reg[3])
    qc.h(ancilla_reg[3])
    qc.cx(ancilla_reg[3], data_reg[2])
    qc.cx(ancilla_reg[3], data_reg[3])
    qc.cx(ancilla_reg[3], data_reg[4])
    qc.h(ancilla_reg[3])
    qc.measure(ancilla_reg[3], syndrome_reg[3])
    # qc.reset(ancilla_reg[3]) # No need to reset the last one usually

# --- Simulation 1: Measure Syndrome BEFORE Error ---

print("--- Running Simulation: BEFORE Error ---")
qc_before = QuantumCircuit(data, ancilla, syndrome)

# Step 1: Measure stabilizers (Ancillas start at |0>, Data qubits start at |0>)
measure_stabilizers(qc_before, data, ancilla, syndrome)

# Step 2: Simulate
simulator = AerSimulator()
job_before = simulator.run(qc_before, shots=1024)
result_before = job_before.result()
counts_before = result_before.get_counts()

# Step 3: Print and Visualize Results
print("Syndrome measurements (s[3]s[2]s[1]s[0]):", counts_before)
print("Expected 'before' (no error): {'0000': 1024}")
# The order is S4 S3 S2 S1 based on the measurement order in the function
# plot_histogram(counts_before, title='Syndrome Before Error')
# plt.show() # Display the plot

# --- Simulation 2: Measure Syndrome AFTER Error ---

print("\n--- Running Simulation: AFTER X Error on D3 ---")
qc_after = QuantumCircuit(data, ancilla, syndrome)

# Step 1: Introduce X error on D3 FIRST
qc_after.x(data[2])  # Bit flip on D3
qc_after.barrier() # Visual separator

# Step 2: Measure stabilizers
measure_stabilizers(qc_after, data, ancilla, syndrome)

# Step 3: Simulate
# We can reuse the simulator object
job_after = simulator.run(qc_after, shots=1024)
result_after = job_after.result()
counts_after = result_after.get_counts()

# Step 4: Print and Visualize Results
print("Syndrome measurements (s[3]s[2]s[1]s[0]):", counts_after)
# Explanation of expected result:
# X error on D3 anti-commutes with Z-type stabilizers involving D3.
# S2 = Z_D1 Z_D3 Z_D4 -> Anti-commutes -> Measurement flips to 1 (s[1])
# S3 = Z_D2 Z_D3 Z_D5 -> Anti-commutes -> Measurement flips to 1 (s[2])
# X error commutes with X-type stabilizers S1 and S4.
# S1 = X_D1 X_D2 X_D3 -> Commutes -> Measurement stays 0 (s[0])
# S4 = X_D3 X_D4 X_D5 -> Commutes -> Measurement stays 0 (s[3])
# So, the syndrome string s[3]s[2]s[1]s[0] should be 0110.
print("Expected 'after' (X on D3): {'0110': 1024}") # Note: Corrected expected value!
# plot_histogram(counts_after, title='Syndrome After X Error on D3')
# plt.show() # Display the plot

# --- Optional: Draw the circuits ---
# print("\nCircuit Before Error:")
# print(qc_before.draw(output='text'))
# print("\nCircuit After Error:")
# print(qc_after.draw(output='text'))

--- Running Simulation: BEFORE Error ---
Syndrome measurements (s[3]s[2]s[1]s[0]): {'1000': 263, '0000': 224, '0001': 259, '1001': 278}
Expected 'before' (no error): {'0000': 1024}

--- Running Simulation: AFTER X Error on D3 ---
Syndrome measurements (s[3]s[2]s[1]s[0]): {'0110': 272, '1111': 241, '0111': 243, '1110': 268}
Expected 'after' (X on D3): {'0110': 1024}


In [25]:
from qiskit.result import marginal_counts
print(marginal_counts(counts_before, [1, 2]))  # s[1]s[2]
print(marginal_counts(counts_after, [1, 2]))

{'00': 1024}
{'11': 1024}
